**Section 4: Linear Regression - Solution**

Notebook for "Introduction to Data Science and Machine Learning"

version 1.0, May 27 2024

# Linear Regression

In this lab we will use functions and carry out a linear regression task. 

We need the following modules. So please run the following code:

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

## 1. Loading the data

We load the dataset. We will use one of the data sets that is often recommended for classification tasks.

In [ ]:
data=pd.read_csv('data/winequality-red.csv')

## 2. Getting to know the data

Use code you used in former lab assignments:

- Display the first five rows of the data

In [ ]:
# your code


- Display the information about the data 

In [ ]:
# your code


- Display the statistical information of the numerical data

In [ ]:
# your code


This dataset has many features, we will thus solve a **Multivariable Linear Regression** problem.

## 3. Data cleaning

Let's check whether there is any missing data: 

In [ ]:
data.isna().sum()

As we could see above there are no null values. We should now check for duplicate entries. 


Lets count how many duplicated rows are in the data set. `duplicated()` returns a boolean series denoting duplicate rows.

In [ ]:
# Number of duplicates
data.duplicated().sum()

We use `inplace=True` in `drop_duplicates()` to modify the data by removing the duplicates, i.e. we modify the data frame. 

*Please remember that we should do this always with the greatest care!*

In [ ]:
# drop the duplicates in the original data
data.drop_duplicates(inplace=True)

Before we split the data, we take a look at the `quality` attribute. Contrary to the other atrributes it is of type `int32` with few unique values:

In [ ]:
data.quality.unique()

The `quality` can be seen as a target variable / class. We will not use it in the regression task and thus drop it.

In [ ]:
regressionData=data.drop(columns=['quality'])

## 4. Data Splitting

In machine learning we normally split our data into several sets:
- the training set is used for training
- the test set consists of data that have not been used for training and is used to evaluate the quality of the learned model
- if there are different parameters to choose from an additional set, a validation set, is used to select the best among different models that is then tested with an idependet data set.

The easiest split is splitting the data into 2 sets, the *training* and the *test* set. Data in the training set is used to learn a model (here the linear regression). We then use the test set data (that is unknown to the model) to estimate how good the model is. We will look at this in the lecture. 

We use the function `train_test_split()` to create random trainings and test subsets.

Often the data is split in 70% trainings and 30% test data.

Using `random_state` we get reproducible splits.

**Question:** 

Do you have an idea why "reproducibility" might be important?

**Your answer**

Now we split the data set into a trainings and a test data set.

In [ ]:
from sklearn.model_selection import train_test_split

dfTrain, dfTest=train_test_split(regressionData,train_size=0.7,test_size=0.3,random_state=12)

Let's take a look at the data set sizes:

In [ ]:
print('training set:',dfTrain.shape[0])
print('test set:',dfTest.shape[0])

Let's plot the data:

In [ ]:
# uncomment this code if you want to see the pairplots (it takes some time to display the graph)
#sns.pairplot(dfTrain)

And the correlation matrix.

In [ ]:
sns.heatmap(data.corr(),cmap='RdBu') # with annot=True the values will be displayed

## 5. Scaling

As the range of the values of the features are different we apply scaling. We use the `MinMaxScaler` to perform scaling to the interval $[0,1]$. 

**Important:** It is important that the training, i.e. the development of the regression model, is completely **independent** of the test data. Therefore, the scaling must **only** base on the trainings data. It is thus performed after splitting the data. 

We use the `MinMaxScaler` of `scikit-learn` to perform the scaling. `MinMaxScaler` is a class. We first create a so-called instance of this class, an object, and then we use the object to perform the scaling. The following code shows one example how to apply the `MinMaxScaler`:

In [ ]:
# import the class
from sklearn.preprocessing import MinMaxScaler

# instantiate (=create) a MinMaxScaler object with default characteristics
theScaler=MinMaxScaler()

# determine minimum and maximum to be used for scaling (initialize the scaler for the 
# data)
theScaler.fit(dfTrain)

Now we can apply the Scaler:

In [ ]:
# scale the data to [0,1] and store it in a numpy array
dfTrainScaled=theScaler.transform(dfTrain)

# scale the data to [0,1] and replace the values in the data frame 
dfTrain[:]=theScaler.transform(dfTrain[:])

`dfTrainScaled` is a `numpy` array while `dfTrain` is still a data frame. 

In [ ]:
print(type(dfTrainScaled),type(dfTrain))

Please display the statistical information of `dfTrain`.

In [ ]:
# your code

Please apply the same scaler model to the test data, and replace the values in the `dfTest` data frame. 

In [ ]:
# your code

Now let's take a look at the minimum and maximum values of the columns in the testing and the trainigs data frame.

In [ ]:
minMaxDf=pd.DataFrame([dfTest.min(),dfTrain.min(),dfTest.max(),dfTrain.max()], 
                      index=["test min","train min", "test max","train max"]).T 

minMaxDf

**Question:** Let's take look at the values:
- Is there anything unexpected?
- Can you explain the values?

**Answer:**

## 6. Regression

The columns of the data frame are:

In [ ]:
dfTrain.columns

We will build a model for `citric acid`.

We first create the `X` and `y` data from `dfTrain`:

In [ ]:
# copy the data frame to a new one
XTrain=dfTrain.copy()

# remove the column 'citric acid' from the data frame and store it in a new variable
yTrain=XTrain.pop('citric acid')

print('shape XTrain',XTrain.shape)
print('shape yTrain',yTrain.shape)

We now build a linear regression model using the class `LinearRegression` in the module `sklearn.linear_model`. As with the `MinMaxScaler` we need to create an instance first.

In [ ]:
# import the class
from sklearn.linear_model import LinearRegression

# instantiate the class, create a regression model
lm=LinearRegression()


And then we train the model:

In [ ]:
# train the model
lm.fit(XTrain,yTrain)

Let's display the names of the features used during the training process:

In [ ]:
lm.feature_names_in_

Now we can display the coefficients that were learned:

In [ ]:
lm.coef_

We now use the model to predict data in the test data set. First we need to remove the `citric acid` column from the data:

In [ ]:
# copy the data frame to a new one
XTest=dfTest.copy()

# remove the column 'citric acid' from the data frame and store it in a new variable
yTest=XTest.pop('citric acid')

print('shape Xtest',XTest.shape)
print('shape yTest',yTest.shape)

And now we use the model to predict the citric acid data:

In [ ]:
yPredicted=lm.predict(XTest)

And now we can calculate the error and plot them:

In [ ]:
errors=yTest-yPredicted

plt.boxplot(errors,vert=False)

As we have a multivariate linear regression, with 10 features we cannot draw a scatterplot with the residuals as in the "Gradient Descent" Lab. 

In the following plot we thus simply plot the number of the data set on the $x$-axis.

In [ ]:
xNumbers=list(range(1,len(yTest)+1))
plt.plot(xNumbers,yTest,'.',label='the data')
plt.plot(xNumbers,yPredicted,'.',label='predicted')
plt.legend()

Our model does not seem to be very good. Let's take a look at the stastics.

## 7. Statistics of Linear Regression

In order to take a look at the statistics of the regression we build the model using the `statsmodels.api`.

In [ ]:
import statsmodels.api as sm 

First we add the constant ($x_0$ that is always 1) as an additional feature for the multivariate regression.

In [ ]:
# create a copy of the data frame
XTrainDF=XTrain.copy()
# add the constant
XTrainDF = sm.add_constant(XTrainDF)

Then we train the model using ordinary least square:

In [ ]:
smLm = sm.OLS(yTrain,XTrainDF).fit()

And we display the statistics:

In [ ]:
print(smLm.summary())

For more information on the statistics please take a look at the document in moodle.

In above table we see the $R^2$ value of 0.690. The adjusted $R^2$ value is 0.687. 

You might remember that some variables might improve the result of linear regression by chance. Therefore $R^2$ is corrected (adjusted $R²$).


## 8. Feature Selection

Above model bases on 10 features. 

Let's try to reduce the number of features: 

First we develop a model with 8 features. Of course we should not simply select any eight features but the ones that are most important for the model.

We use the class `RFE`(recursive feature elimination) in the module `sklearn.feature_selection`.

In [ ]:
from sklearn.feature_selection import RFE

Now we instantiate the class using the linear regression model we built and asking for  the $n$ most important features. We want to look for the most important 8 features and thus select $n=8$. 

We create a new estimator object. This object takes the traines linear regression model (`LinearRegression`) and the number of features we are interested in as parameters:

In [ ]:
rfe=RFE(lm,n_features_to_select=8)

And now we apply this estimator object to our trainings data:

In [ ]:
rfeObj=rfe.fit(XTrainDF, yTrain)

And display the result

In [ ]:
print("{:22} {:7} {:2}".format("column","support","rank"))
for el in list(zip(XTrainDF.columns,rfeObj.support_,rfeObj.ranking_)):
    print(f"{el[0]:22} {str(el[1]):7} {el[2]:2}")

We display the support as well as the ranking. The 8 most important features are ranked with 1 and have a support of `True`. We use now this support to reduce the features to the most relevant 8:

In [ ]:
XTrainDF2=XTrainDF[XTrainDF.columns[rfe.support_]]

print(XTrainDF2.columns)


The features might equally have been reduced using `XTrainDF3=rfeObj.transform(XTrainDF)`. The result in this case would be a `numpy` array.

Now we can relearn the model and calculate the new statistics:

In [ ]:
XTrainDF2 = sm.add_constant(XTrainDF2)
smLm2 = sm.OLS(yTrain,XTrainDF2).fit()
print(smLm2.summary())

Our model has the same $R^2$ value.

## 9.Variance Inflation Factor

When we reduce feature we try to identify the most important ones. If two features have a high linear correlation the model becomes complex and very hard to be interpreted. Therefore, it is sometimes advisable to eliminate one of the two correlating features. The `variance_inflation_factor` helps identifying such features:

First, we import the required class:

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

Then we create a data frame with:
- the column names
- the variance inflation factor for the columns


In [ ]:
# calculate the VIF values
vifList=[]
dfAsNumpy=XTrainDF2.values
# for all columns
for c in range(XTrainDF2.shape[1]):
    vifList.append(variance_inflation_factor(dfAsNumpy,c))
vifArray=np.array(vifList)
vifArray=vifArray.round(2)

# create an empty data frame
vif = pd.DataFrame()

# create a column features with column names    
vif['Features'] = XTrainDF2.columns
# create a column VIF with the calculated values
vif['VIF']=vifArray
# sort the data frame in descending order
vif = vif.sort_values(by = "VIF", ascending = False)

print(vif)

It is recommended to eliminate any feature with an VIF larger than 5 (https://www.statsmodels.org/stable/generated/statsmodels.stats.outliers_influence.variance_inflation_factor.html#). We do not have such a feature (remember that const is the intercept).

## 10. Exercise

Repeat the steps in section 6 ff. to create a regression model for `residual sugar`. Do you get better results? How many features should you take into account?

In [ ]:
# Your Code

<a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-nd/4.0/88x31.png" /></a><br />This notebook was created by Christina B. Class for teaching at EAH Jena and is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-nd/4.0/">Creative Commons Attribution-NonCommercial-NoDerivatives 4.0 International License</a>.